- 除了弹性分布式数据集RDD外，Spark的第二种低级API是分布式共享变量
- 分布式共享变量包括两种类型：广播变量（broadcast variable）和累加器（accumulator）
- 累加器：将所有任务中的数据累加到一个共享变量中
- 广播变量：允许在所有工作节点上保存一个共享值，以便在Spark操作时重用

<h4>广播变量</h4>

- 在驱动节点上使用变量的一般方法是简单地在函数闭包中引用（function closure）
- 但是这种方式效率很低，，因为当在闭包（closure）中使用变量时，必须在工作节点上执行多次反序列化
- 此外，如果在多个Spark操作和作业中使用相同的变量，它将被重复发送到工作节点的每一个作业中


- 以上是引入广播变量的原因
- 广播变量是共享的、不可修改的变量
- 它缓存在集群中的每个节点上，而不是在每个任务中反复序列化

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Python").getOrCreate()

Exception: Java gateway process exited before sending its port number

In [ ]:
my_collection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
words = spark.sparkContext.parallelize(my_collection, 2)

- 比如，我们希望用其他信息补充单词列表，这个信息可能很大，这时候我们就要考虑将这个信息变为广播变量

In [3]:
supplementalData = {
    "Spark": 1000,
    "Definitive": 200,
    "Big": -300,
    "Simple": 100,
}

- 接下来使用spark.sparkContext.broadcast将字典变为广播变量

In [ ]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)

- 可以通过value属性直接访问suppBroadcast的值，该方法在序列化函数是可以直接访问的
- 无需对数据进行序列化，节省大量序列化和反序列化的成本

In [ ]:
suppBroadcast.value

- 下面的例子就用到广播变量为RDD进行转换
- 根据RDD的word查找广播变量的value，如果没有就用0替代

In [ ]:
words.map(lambda word: (word, suppBroadcast.value.get(word, 0)))\
.sortBy(lambda wordPair: wordPair[1]).collect()

<h4>累加器</h4>

- Spark第二种类型的共享变量就是累加器
- 它用于将转换操作更新的值以高效和容错的方式传输到驱动节点
- 累加器提供累加用的变量，Spark集群可以以按行方式对其进行安全更新
- 累加器也遵循Spark的惰性评估机制，如果RDD的某个操作要更新累加器，则它的值只会在实际计算RDD时更新

<h4>简单的总结</h4>

- 广播变量是驱动器把变量广播给所有执行节点
- 累加器是所有执行节点把变量累加到驱动器

<h4>累加器的命名</h4>

- 累加器可以使命名和未命名的，明明累加器将在Spark用户界面显示运行结果，而未命名累加器则不会显示

- 以下例子计算往返中国的航班总数
- 首先导入数据

In [ ]:
flights = spark.read.parquet("./data/flight-data/parquet/2010-summary.parquet")

- 创建名为accChina的累加器

In [ ]:
accChina = spark.sparkContext.accumulator(0)

- Scala两种方法命名累加器
- 第一种
    - val accChina = spark.sparkContext.longAccumulator("China")
- 第二种
    - val accChina = new LongAccumulator
    - spark.sparkContext.register(accChian, "China")

- 下一步定义递增累加器的逻辑

In [1]:
def accChinaFunc(flight_row):
    destination = flight_row["DEST_COUNTRY_NAME"]
    origin = flight_row["ORIGIN_COUNTRY_NAME"]
    if destination == "China":
        accChina.add(flight_row["count"])
    if origin == "China":
        accChina.add(flight_row["count"])

- 通过foreach方法遍历航班数据集的每一行，foreach是一个动作操作

In [ ]:
flights.foreach(lambda flight_row: accChinaFunc(flight_row))

- 和广播变量一样，可以通过value来访问累加器的值

In [ ]:
accChina.value

<h4>自定义累加器</h4>

- 使用Python，可以通过定义AccumulatorParam的子类来创建自定义累加器